In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://www.w3schools.com/php/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [4]:
#printing the staus of the requests
print(status)

200


In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left"><span class="left_h2">PHP</span> Tutorial</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> Forms</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> Advanced</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> OOP</h2>,
 <h2 class="left"><span class="left_h2">MySQL</span> Database</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> XML</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> - AJAX</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> Examples</h2>,
 <h2 class="left"><span class="left_h2">PHP</span> Reference</h2>]

In [7]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

PHP Tutorial
PHP Forms
PHP Advanced
PHP OOP
MySQL Database
PHP XML
PHP - AJAX
PHP Examples
PHP Reference


In [8]:
# contents in html
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

In [13]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/php/"
course_url = "https://www.w3schools.com/php/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
           
            if extra is not None :
                print(extra)
                if extra[0] == '/':
                    base_url = "https://www.w3schools.com"
                    
            elif extra is None :
                continue 
           
                
                
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate htmlHigh')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

default.asp
php_intro.asp
php_install.asp
php_syntax.asp
php_comments.asp
php_variables.asp
php_echo_print.asp
php_datatypes.asp
php_string.asp
php_numbers.asp
php_math.asp
php_constants.asp
php_operators.asp
php_if_else.asp
php_switch.asp
php_looping.asp
php_functions.asp
php_arrays.asp
php_superglobals.asp
php_regex.asp
php_forms.asp
php_form_validation.asp
php_form_required.asp
php_form_url_email.asp
php_form_complete.asp
php_date.asp
php_includes.asp
php_file.asp
php_file_open.asp
php_file_create.asp
php_file_upload.asp
php_cookies.asp
php_sessions.asp
php_filter.asp
php_filter_advanced.asp
php_callback_functions.asp
php_json.asp
php_exceptions.asp
php_oop_what_is.asp
php_oop_classes_objects.asp
php_oop_constructor.asp
php_oop_destructor.asp
php_oop_access_modifiers.asp
php_oop_inheritance.asp
php_oop_constants.asp
php_oop_classes_abstract.asp
php_oop_interfaces.asp
php_oop_traits.asp
php_oop_static_methods.asp
php_oop_static_properties.asp
php_namespaces.asp
php_iterables.asp
php_

In [14]:
length = len(course_links)
length

106

In [15]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(127, 127, 106, 106, 106, 106, 106, 106, 106)

In [16]:
from csv import writer

# writing the file
with open('php_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('php_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,PHP Tutorial,PHP HOME,https://www.w3schools.com/php/default.asp,"PHP is a server scripting language, and a powe...",Learn PHP,NaN,NaN,NaN,"[['\n <!DOCTYPE html>', '\n <html>', '\n <b..."
1,PHP Tutorial,PHP Intro,https://www.w3schools.com/php/php_intro.asp,PHP code is executed on the server.|||Before y...,What You Should Already Know|||What is PHP?|||...,HTML@@@CSS@@@JavaScript|||PHP is an acronym fo...,NaN,NaN,[]
2,PHP Tutorial,PHP Install,https://www.w3schools.com/php/php_install.asp,"To start using PHP, you can:|||If your server ...",PHP Online Compiler / Editor,Find a web host with PHP and MySQL support@@@I...,NaN,NaN,[]
3,PHP Tutorial,PHP Syntax,https://www.w3schools.com/php/php_syntax.asp,"A PHP script is executed on the server, and th...",NaN,NaN,NaN,NaN,[]
4,PHP Tutorial,PHP Comments,https://www.w3schools.com/php/php_comments.asp,A comment in PHP code is a line that is not ex...,Example,Let others understand your code@@@Remind yours...,NaN,NaN,[]


In [17]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [18]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)

In [19]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [21]:
# viewing the top 10 rows
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,PHP Tutorial,PHP HOME,https://www.w3schools.com/php/default.asp,"PHP is a server scripting language, and a powe...",,,,,"[['\n <!DOCTYPE html>', '\n <html>', '\n <b...",Learn PHP
1,PHP Tutorial,PHP Intro,https://www.w3schools.com/php/php_intro.asp,PHP code is executed on the server.|||Before y...,What You Should Already Know|||What is PHP?|||...,HTML@@@CSS@@@JavaScript|||PHP is an acronym fo...,,,[],
2,PHP Tutorial,PHP Install,https://www.w3schools.com/php/php_install.asp,"To start using PHP, you can:|||If your server ...",PHP Online Compiler / Editor,Find a web host with PHP and MySQL support@@@I...,,,[],
3,PHP Tutorial,PHP Syntax,https://www.w3schools.com/php/php_syntax.asp,"A PHP script is executed on the server, and th...",,,,,[],
4,PHP Tutorial,PHP Comments,https://www.w3schools.com/php/php_comments.asp,A comment in PHP code is a line that is not ex...,Example,Let others understand your code@@@Remind yours...,,,[],
5,PHP Tutorial,PHP Variables,https://www.w3schools.com/php/php_variables.asp,"Variables are ""containers"" for storing informa...",,"A variable starts with the $ sign, followed by...",,,"[['\n<?php', '\n$txt = ""Hello world!"";', '\n$x...",
6,PHP Tutorial,PHP Echo / Print,https://www.w3schools.com/php/php_echo_print.asp,"With PHP, there are two basic ways to get outp...",,,,,[],
7,PHP Tutorial,PHP Data Types,https://www.w3schools.com/php/php_datatypes.asp,"Variables can store data of different types, a...",PHP Array|||PHP NULL Value,String@@@Integer@@@Float (floating point numbe...,,,[],
8,PHP Tutorial,PHP Strings,https://www.w3schools.com/php/php_string.asp,"A string is a sequence of characters, like ""He...",,,,,[],PHP String Functions
9,PHP Tutorial,PHP Numbers,https://www.w3schools.com/php/php_numbers.asp,In this chapter we will look in depth into Int...,Example,An integer must have at least one digit@@@An i...,,,[],
